# Complete Black-Litterman Optimization

| n=4 $returns \sim \mathcal{N}(\mu,\sigma)$ Correlation from 0.8-0.9  | n=4 $returns \sim N(\mu,\sigma)$ Independent  | n=4 $returns \sim \mathcal{N}(\mu, sigma)$ Negative correlation (-0.8 to -0.5)  |
|----------------------------------------------------------------------|-----------------------------------------------|---------------------------------------------------------------------------------|
| n=8 $returns \sim \mathcal{N}(\mu,\sigma)$ Correlation from 0.8-0.9  | n=8 $returns \sim N(\mu,\sigma)$ Independent  | n=8 $returns \sim \mathcal{N}(\mu, sigma)$ Negative correlation (-0.8 to -0.5)  |
| n=16 $returns \sim \mathcal{N}(\mu,\sigma)$ Correlation from 0.8-0.9 | n=16 $returns \sim N(\mu,\sigma)$ Independent | n=16 $returns \sim \mathcal{N}(\mu, sigma)$ Negative correlation (-0.8 to -0.5) |

using the method laid out in 4.2, Black-Litterman portfolio optimization

using https://python-advanced.quantecon.org/black_litterman.html

## Read Packages

In [44]:
import numpy as np
import pandas as pd
import scipy.stats as stat
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline
import random
import trace
import sklearn.covariance

In [3]:
import pypfopt as pyp
import warnings
warnings.filterwarnings("ignore")


## Define Functions

Function that Calculated equillibrium values 

This step generates the sample

In [60]:
def bl(n_assets, n_obs, return_vec):
    '''
    This function evaluates the equillibrium returns of a portfolio and generates the sample
    Returns: 
    weights: optimal weights
    bl_returns: BL returns
    S: BL risk
    '''
    market_prices = pd.Series(np.random.randn(n_obs))
    rng = np.random.default_rng()
    cov_struct = rng.multivariate_normal(np.zeros(n_assets), cov = r, size = n_obs)
    S = pyp.risk_models.CovarianceShrinkage(cov_struct).ledoit_wolf()
    delta = pyp.black_litterman.market_implied_risk_aversion(market_prices, risk_free_rate=0.5)

    market_prior = pd.Series(np.random.randn(n_assets))
    view = pd.Series(np.ones(n_assets)*(1/n_assets))
    
    bl = pyp.BlackLittermanModel(S, pi=market_prior, absolute_views=view)
    bl_return = bl.bl_returns()

    ef = pyp.EfficientFrontier(bl_return, r)
    bl.bl_weights(delta)
    weights = bl.clean_weights()

    S_bl = bl.bl_cov()
    return weights, bl_return, S_bl

In [9]:
def one_corr_optimization(n_assets, n_obs, r):
    '''
    First, simulates portfolios then optimizes. 
    This does 100 replications
    '''
    weight_res = np.zeros((99,n_assets))
    return_res = np.zeros((99,n_assets))
    risks_res = []
    
    for i in range(99):
        n_portfolios = 500
        np.random.seed(i)
        rng = np.random.default_rng()
        return_vec = rng.multivariate_normal(np.zeros(n_assets), cov = r, size = n_obs)
        weights, returns, risks = bl(n_assets, n_obs, return_vec) 
        weights = list(weights.items())
        w = [x[1] for x in weights]
        weight_res[i,:] = w
        return_res[i,:] = np.array(returns).T
        risks_res.append([risks])
    return weight_res, return_res, risks_res

In [6]:
def one_corr_optimization_zeros(n_assets, n_obs, r):
    '''
    First, simulates portfolios then optimizes. 
    This does 100 replications
    '''
    weight_res = np.zeros((99,n_assets))
    return_res = np.zeros((99,n_assets))
    risks_res = []
    
    for i in range(99):
        np.random.seed(i)
        rng = np.random.default_rng()
        return_vec = rng.multivariate_normal(np.zeros(n_assets), cov = r, size = n_obs)
        _75_perct = int(len(return_vec[:,0])*3/4)
        return_vec[random.sample(list(range(1000)), _75_perct),0] = 0
        np.random.seed(i*5)
        return_vec[random.sample(list(range(1000)), _75_perct),1] = 0
        
        weights, returns, risks = bl(n_assets, n_obs, return_vec)
        weights = list(weights.items())
        w = [x[1] for x in weights]
        weight_res[i,:] = w
        return_res[i,:] = np.array(returns).T
        risks_res.append([risks])
    return weight_res, return_res, risks_res

# Perform Optimization
## Correlated

## $n=4$

In [49]:
n_assets = 4
n_obs = 1000
r = np.random.uniform(0.8, 0.9, (n_assets,n_assets))
np.fill_diagonal(r, 1)

In [55]:
n4_weights_corr, n4_returns_corr, n4_risks_corr = one_corr_optimization(n_assets, n_obs, r) #
np.round(n4_weights_corr.mean(axis=0),3)

array([0.15 , 0.003, 0.75 , 0.098])

In [56]:
np.round(n4_weights_corr.std(axis=0), 3)

array([4.553, 4.087, 2.32 , 2.742])

In [271]:
np.array(n4_risks_corr).shape

(99, 1, 4, 4)

In [54]:
n4_weights_corr_os, n4_returns_corr_os, n4_risks_corr_os = one_corr_optimization_zeros(n_assets, n_obs, r)
np.round(n4_weights_corr_os.mean(axis=0),3)

array([ 0.196,  0.201, -0.935,  1.537])

In [57]:
np.round(n4_weights_corr_os.std(axis=0), 3)

array([13.198,  4.555,  8.213, 11.316])

## $ n = 8$

In [61]:
n_assets = 8
n_obs = 1000
r = r = np.random.uniform(0.8, 0.9, (n_assets,n_assets))
np.fill_diagonal(r, 1)

In [62]:
n8_weights_corr, n8_returns_corr, n8_risks_corr = one_corr_optimization(n_assets, n_obs, r) 
np.round(n8_weights_corr.mean(axis=0),3)

array([ 0.041, -0.21 ,  0.166,  0.386,  0.056, -0.244,  0.046,  0.759])

In [63]:
np.round(n8_weights_corr.std(axis=0),3)

array([2.97 , 2.822, 1.121, 4.244, 1.31 , 2.973, 2.218, 5.21 ])

In [64]:
n8_weights_corr_os, n8_returns_corr_os, n8_risks_corr_os = one_corr_optimization_zeros(n_assets, n_obs, r)
np.round(n8_weights_corr_os.mean(axis=0),3)

array([ 0.003, -0.195,  0.036,  0.187,  0.149,  0.008, -0.174,  0.986])

In [65]:
np.round(n8_weights_corr_os.std(axis=0),3)

array([5.018, 3.012, 2.578, 4.744, 1.638, 4.412, 3.515, 5.753])

## $n=16$

In [66]:
n_assets = 16
n_obs = 1000
r = np.random.uniform(0.8, 0.9, (n_assets,n_assets))
np.fill_diagonal(r, 1)

In [67]:
n16_weights_corr, n16_returns_corr, n16_risks_corr = one_corr_optimization(n_assets, n_obs, r) 
np.round(n16_weights_corr.mean(axis=0),3)

array([ 0.324,  0.382,  0.131, -0.796,  0.221,  0.13 ,  0.216, -0.733,
        1.137,  0.401, -0.355, -0.055,  0.196,  0.525, -0.387, -0.339])

In [68]:
np.round(n16_weights_corr.std(axis=0),3)

array([1.379, 2.568, 2.674, 8.792, 3.297, 2.407, 2.608, 6.112, 7.363,
       3.816, 4.514, 3.171, 4.434, 2.664, 3.107, 4.96 ])

In [69]:
n16_weights_corr_os, n16_returns_corr_os, n16_risks_corr_os = one_corr_optimization_zeros(n_assets, n_obs, r)
np.round(n16_weights_corr_os.mean(axis=0),3)

array([ 0.016,  0.057,  0.224,  0.285,  0.367,  0.038, -0.109, -0.071,
        0.211, -0.059,  0.014,  0.28 , -0.073, -0.149, -0.087,  0.055])

In [70]:
np.round(n16_weights_corr_os.std(axis=0),3)

array([1.542, 1.336, 1.738, 1.201, 1.3  , 1.207, 1.457, 2.153, 1.146,
       1.252, 1.584, 1.454, 1.442, 1.324, 1.139, 1.406])

## Independent

In [71]:
n_assets = 4
n_obs = 1000
r = np.eye(n_assets)

In [72]:
n4_weights_ind, n4_returns_ind, n4_risks_ind = one_corr_optimization(n_assets, n_obs, r) 
np.round(n4_weights_ind.mean(axis=0),3)

array([ 0.06 ,  0.15 ,  0.855, -0.065])

In [74]:
np.round(n4_weights_ind.std(axis=0),3)

array([4.357, 3.681, 2.311, 2.749])

In [78]:
n4_weights_ind_os, n4_returns_ind_os, n4_risks_ind_os = one_corr_optimization_zeros(n_assets, n_obs, r)
np.round(n4_weights_ind_os.mean(axis=0),3)

array([0.011, 0.032, 0.012, 0.945])

In [79]:
np.round(n4_weights_ind_os.std(axis=0),3)

array([12.756,  5.021,  5.471, 12.799])

## $n=8$

In [80]:
n_assets = 8
n_obs = 1000
r = np.eye(n_assets)

In [81]:
n8_weights_ind, n8_returns_ind, n8_risks_ind = one_corr_optimization(n_assets, n_obs, r) 
np.round(n8_weights_ind.mean(axis=0),3)

array([ 0.446, -0.005,  0.07 , -0.128,  0.308, -0.111, -0.003,  0.424])

In [82]:
np.round(n8_weights_ind.std(axis=0),3)

array([2.427, 1.326, 1.067, 2.595, 3.675, 1.899, 2.876, 1.878])

In [83]:
n8_weights_ind_os, n8_returns_ind_os, n8_risks_ind_os = one_corr_optimization_zeros(n_assets, n_obs, r)
np.round(n8_weights_ind_os.mean(axis=0),3)

array([ 0.613, -0.042, -0.137, -0.087,  0.043,  0.058,  0.154,  0.397])

In [84]:
np.round(n8_weights_ind_os.std(axis=0),3)

array([2.941, 1.668, 2.574, 2.297, 1.59 , 1.717, 1.597, 2.198])

## $n=16$

In [89]:
n_assets = 16
n_obs = 1000
r = np.eye(n_assets)

In [90]:
n16_weights_ind, n16_returns_ind, n16_risks_ind = one_corr_optimization(n_assets, n_obs, r) 
np.round(n16_weights_ind.mean(axis=0),3)

array([ 0.173,  0.293,  0.056, -0.796,  0.324, -0.155, -0.101, -0.262,
        0.986,  0.257, -0.648,  0.232,  0.464,  0.503, -0.099, -0.227])

In [91]:
np.round(n16_weights_ind.std(axis=0),3)

array([1.167, 2.668, 2.517, 8.901, 3.341, 1.593, 2.121, 5.743, 7.398,
       3.818, 4.54 , 2.344, 4.051, 2.601, 3.276, 4.823])

In [92]:
n16_weights_ind_os, n16_returns_ind_os, n16_ind_risks_os = one_corr_optimization_zeros(n_assets, n_obs, r)
np.round(n16_weights_ind_os.mean(axis=0),3)

array([ 0.102,  0.26 ,  0.832, -0.769,  0.331,  0.532, -0.021, -0.489,
        0.779,  0.614, -0.541,  0.286, -0.608,  0.229, -0.076, -0.462])

In [93]:
np.round(n16_weights_ind_os.std(axis=0),3)

array([1.864, 2.3  , 3.837, 8.663, 4.355, 3.273, 2.995, 6.391, 7.909,
       4.034, 4.266, 3.837, 7.186, 3.12 , 3.889, 4.897])

## Negatively Correlated

## $n=4$

In [94]:
n_assets = 4
n_obs = 1000
r = r = np.random.uniform(-0.8, -0.5, (n_assets,n_assets))
np.fill_diagonal(r, 1)

In [95]:
n4_weights_neg, n4_returns_neg, n4_risks_neg = one_corr_optimization(n_assets, n_obs, r) 
np.round(n4_weights_neg.mean(axis=0),3)

array([0.03 , 0.166, 0.802, 0.002])

In [99]:
np.round(n4_weights_neg.std(axis=0),3)

array([4.36 , 3.683, 2.36 , 2.768])

In [97]:
n4_weights_neg_os, n4_returns_neg_os, n4_risks_neg_os = one_corr_optimization_zeros(n_assets, n_obs, r)
np.round(n4_weights_neg_os.mean(axis=0),3)

array([-0.987,  0.295,  0.008,  1.685])

In [98]:
np.round(n4_weights_neg_os.std(axis=0),3)

array([ 9.785,  4.512,  4.485, 10.763])

## $n=8$

In [100]:
n_assets = 8
n_obs = 1000
r = r = np.random.uniform(-0.8, -0.5, (n_assets,n_assets))
np.fill_diagonal(r, 1)

In [101]:
n8_weights_neg, n8_returns_neg, n8_risks_neg = one_corr_optimization(n_assets, n_obs, r) 
np.round(n8_weights_neg.mean(axis=0),3)

array([ 0.139,  0.334,  0.313,  0.952,  0.172, -0.462, -0.584,  0.135])

In [103]:
np.round(n8_weights_neg.std(axis=0),3)

array([3.436, 5.399, 2.384, 7.631, 3.782, 3.669, 4.764, 8.791])

In [102]:
n8_weights_neg_os, n8_returns_neg_os, n8_risks_neg_os = one_corr_optimization_zeros(n_assets, n_obs, r)
np.round(n8_weights_neg_os.mean(axis=0),3)

array([-0.036, -0.185,  0.013,  0.079, -0.11 ,  0.122, -0.026,  1.142])

In [104]:
np.round(n8_weights_neg_os.std(axis=0),3)

array([5.233, 3.099, 2.821, 4.891, 2.77 , 4.475, 4.249, 5.862])

## $n=16$

In [105]:
n_assets = 16
n_obs = 1000
r = r = np.random.uniform(-0.8, -0.5, (n_assets,n_assets))
np.fill_diagonal(r, 1)

In [106]:
n16_weights_neg, n16_returns_neg, n16_risks_neg = one_corr_optimization(n_assets, n_obs, r) 
np.round(n16_weights_neg.mean(axis=0),3)

array([ 0.263,  0.673,  0.248, -0.891,  0.128,  0.214,  0.181, -0.804,
        1.003,  0.426, -0.231, -0.139,  0.193,  0.424, -0.297, -0.391])

In [107]:
np.round(n16_weights_neg.std(axis=0),3)

array([1.336, 2.543, 2.687, 8.77 , 3.316, 2.437, 2.6  , 6.081, 7.381,
       3.842, 4.536, 3.171, 4.433, 2.636, 3.038, 4.962])

In [108]:
n16_weights_neg_os, n16_returns_neg_os, n16_risks_neg_os = one_corr_optimization_zeros(n_assets, n_obs, r)
np.round(n16_weights_neg_os.mean(axis=0),3)

array([ 0.217,  0.085,  0.4  , -0.517,  0.514,  0.224,  0.155, -0.711,
        0.905,  0.503, -0.292,  0.091,  0.037,  0.122, -0.293, -0.44 ])

In [109]:
np.round(n16_weights_neg_os.std(axis=0),3)

array([1.71 , 1.886, 2.163, 6.919, 2.415, 2.192, 2.42 , 5.206, 5.816,
       3.315, 3.645, 3.185, 3.776, 2.139, 2.558, 4.005])